In [1]:
from __future__ import print_function 
import os, sys
import numpy as np
from six.moves import cPickle
from collections import OrderedDict

import tensorflow as tf

sys.path.append('..')
from tfomics import neuralnetwork as nn
from tfomics import utils, learn
from model_zoo import simple_genome_model

In [5]:

def load_data(filepath):
    # load training set
    print("loading data from: " + filepath)
    f = open(filepath, 'rb')
    print("loading train data")
    #train = cPickle.load(f)
    train = cPickle.load(f, encoding='latin1')
    print("loading cross-validation data")
    #cross_validation = cPickle.load(f)
    cross_validation = cPickle.load(f, encoding='latin1')
    print("loading test data")
    #test = cPickle.load(f)
    test = cPickle.load(f, encoding='latin1')
    f.close()

    X_train = train[0].transpose((0,2,1))
    y_train = train[1]
    X_valid = cross_validation[0].transpose((0,2,1))
    y_valid = cross_validation[1]
    X_test = test[0].transpose((0,2,1))
    y_test = test[1]
    
    X_train = X_train.reshape(X_train.shape + (1,)) 
    X_train = X_train.transpose([0,1,3,2]).astype(np.float32)
    y_train = y_train.astype(np.int32)

    X_valid = X_valid.reshape(X_valid.shape + (1,)) 
    X_valid = X_valid.transpose([0,1,3,2]).astype(np.float32)
    y_valid = y_valid.astype(np.int32)


    X_test = X_test.reshape(X_test.shape + (1,)) 
    X_test = X_test.transpose([0,1,3,2]).astype(np.float32)
    y_test = y_test.astype(np.int32)

    return X_train, y_train, X_valid, y_valid, X_test, y_test


# load data
filename = 'Localized_N=100000_S=200_M=50_G=20_data.pickle'
data_path = '/Users/juliankimura/Desktop/tensorflow/data'
#data_path = '/Users/juliankimura/Desktop/tensorflow/data'
filepath = os.path.join(data_path,filename)
X_train, y_train, X_valid, y_valid, X_test, y_test = load_data(filepath)

loading data from: /Users/juliankimura/Desktop/tensorflow/data/Localized_N=100000_S=200_M=50_G=20_data.pickle
loading train data
loading cross-validation data
loading test data


In [6]:
X_train.shape

(70000, 200, 1, 4)

In [7]:
# get shapes
num_data, seq_length, _, dim = X_train.shape
input_shape=[None, seq_length, 1, dim]
num_labels = y_train.shape[1]   # number of labels (output units)

# load model
net, placeholders, optimization = simple_genome_model.model(input_shape, num_labels)

# build neural network class
nnmodel = nn.NeuralNet(net, placeholders)
nnmodel.inspect_layers()

# compile neural trainer
save_name = 'test'
save_path = utils.make_directory(data_path, 'results')
save_path = utils.make_directory(save_path, 'simple_genome')
filepath = os.path.join(save_path, save_name)
nntrainer = nn.NeuralTrainer(nnmodel, optimization, save='best', filepath=filepath)

NameError: name 'inputs' is not defined

In [ ]:
train = {'inputs': X_train, 'targets': y_train, 'keep_prob': 0.8, 'is_training': True}
valid = {'inputs': X_valid, 'targets': y_valid, 'keep_prob': 1, 'is_training': False}
data = {'train': train, 'valid': valid}
learn.train_minibatch(nntrainer, data, batch_size=128, num_epochs=500, 
                        patience=10, verbose=1, shuffle=True)

In [ ]:
test = {'inputs': X_test, 'targets': y_test, 'keep_prob': 1, 'is_training': False}
test_loss = nntrainer.test_model(test, batch_size=512)

In [ ]:
nntrainer.close_sess()